In [ ]:
import numpy as np
import pandas as pd


In [ ]:
import os
dataset_path = os.path.join(os.getcwd(),'Dataset')
datasets=[]
for filename in os.listdir(dataset_path):
    datasets.append(os.path.join(dataset_path,filename))
print(datasets)

In [ ]:
sample = pd.read_csv(datasets[3])
spray = pd.read_csv(datasets[4])
test = pd.read_csv(datasets[5])
train = pd.read_csv(datasets[6])
weather = pd.read_csv(datasets[7])

In [ ]:
# train.columns
# train.info()
# train.describe
print(train.isna().sum(),test.isna().sum())
#no null data points


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#visual EDA
sns.scatterplot(data=train,x='Latitude',y='Longitude',size='NumMosquitos',sizes=(1,100))

In [ ]:
sns.scatterplot(data=train,x='Latitude',y='Longitude',hue='WnvPresent',size='NumMosquitos')

In [ ]:
sns.scatterplot(data=train,x='Latitude',y='Longitude',hue='Species')

In [ ]:
#explore by species
num_wnv = train.loc[train['WnvPresent']==1]['NumMosquitos'].sum()
num_tot = train['NumMosquitos'].sum()
print(f'Percentage of WNV for all mosquitos: {num_wnv/num_tot*100}%')

In [ ]:
train.groupby(['Species','WnvPresent'])['NumMosquitos'].sum()
#erraticus, salinarius, tarsalis, and territans have low number of data points

In [ ]:
# train['Address'].apply(lambda st: st[st.find("IL")+1:st.find(", USA")]).unique()
train['zipcode'] = train['Address'].str.extract(r'(\d{5}\-?\d{0,4})')

In [ ]:
#find missing zipcodes

In [165]:
import geopy
def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']['postcode']


geolocator = geopy.Nominatim(user_agent='geoapiExercises')

df=train.loc[train['zipcode'].isna()]
zipcodes = df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Latitude', lon_field='Longitude')

fixed_train = train.loc[train['zipcode'].isna()]
fixed_train['zipcode']=zipcodes
fixed_train.to_csv('train_miss_zipcodes.csv')

train.iloc[fixed_train.index]=fixed_train

In [177]:
train.iloc[fixed_train.index]=fixed_train

In [184]:
train.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'zipcode'],
      dtype='object')

In [191]:
train2 = train.drop(['Address','Block','AddressNumberAndStreet','AddressAccuracy'],axis=1)
train2

,Date,Species,Street,Trap,Latitude,Longitude,NumMosquitos,WnvPresent,zipcode
0,2007-05-29,CULEX PIPIENS/RESTUANS,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,60634
1,2007-05-29,CULEX RESTUANS,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,60634
2,2007-05-29,CULEX RESTUANS,N MANDELL AVE,T007,41.994991,-87.769279,1,0,60646
3,2007-05-29,CULEX PIPIENS/RESTUANS,W FOSTER AVE,T015,41.974089,-87.824812,1,0,60656
4,2007-05-29,CULEX RESTUANS,W FOSTER AVE,T015,41.974089,-87.824812,4,0,60656
...,...,...,...,...,...,...,...,...,...
10501,2013-09-26,CULEX PIPIENS/RESTUANS,W 72ND ST,T035,41.763733,-87.742302,6,1,60638
10502,2013-09-26,CULEX PIPIENS/RESTUANS,N RIDGE AVE,T231,41.987280,-87.666066,5,0,60660
10503,2013-09-26,CULEX PIPIENS/RESTUANS,N ASHLAND AVE,T232,41.912563,-87.668055,1,0,60622
10504,2013-09-26,CULEX PIPIENS/RESTUANS,N HARLEM AVE,T233,42.009876,-87.807277,5,0,60631


In [221]:
res=set()
weather.CodeSum.str.split().apply(res.update)
for i in res:
    weather[i] = weather.CodeSum.apply(lambda x: 1 if i in x else 0)
    
weather['Sunrise'] = weather['Sunrise'].replace('-',np.nan).ffill()
weather['Sunset'] = weather['Sunset'].replace('-',np.nan).ffill()

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed', 'MIFG', 'FU', 'FG', 'BR',
       'VCFG', 'BCFG', 'SQ', 'DZ', 'FG+', 'TSRA', 'GR', 'RA', 'TS', 'SN',
       'VCTS', 'HZ'],
      dtype='object')

In [227]:
weather.loc[weather['Station']==2]['Sunrise'].unique()

array(['-'], dtype=object)

In [243]:
weather['Sunrise'] = weather['Sunrise'].replace('-',np.nan)\
                                        .ffill()\
                                        .apply(int)/2400

weather['Sunset'] = weather['Sunset'].replace('-',np.nan)\
                                        .ffill()\
                                        .apply(int)/2400

weather2 = weather.drop(['Depart','Depth','Water1','SnowFall','PrecipTotal'])                                        

0       0.186667
1       0.186667
2       0.186250
3       0.186250
4       0.185833
          ...   
2939    0.258333
2940    0.259167
2941    0.259167
2942    0.259583
2943    0.259583
Name: Sunrise, Length: 2944, dtype: float64